This notebook contains code to pull all the required data for this project locally into csv files.

In [7]:
import yfinance as yf
import pandas as pd
import numpy as np


# Example ETF: TLT (20+ Year Treasury ETF)
etf = yf.Ticker('TLT')


# Get daily historical prices (adjusted close, open, high, low, volume)
etf_hist = etf.history(period='5y', interval='1d')


# Keep relevant columns
etf_hist = etf_hist[['Open', 'High', 'Low', 'Close', 'Volume']].rename(
columns={'Close':'etf_close', 'Open':'etf_open', 'High':'etf_high', 'Low':'etf_low', 'Volume':'etf_volume'})


# Save
etf_hist.to_csv('../data/raw/tlt_daily.csv')

Download US Treasury yield data from FRED (Federal Reserve Economic Data)

In [3]:
import pandas_datareader.data as web


# Daily yields for 20Y and 30Y
start = '2018-01-01'
end = '2023-12-31'
yields = web.DataReader(['DGS20', 'DGS30'], 'fred', start, end)
yields.rename(columns={'DGS20':'yield_20y', 'DGS30':'yield_30y'}, inplace=True)


# Save
yields.to_csv('../data/raw/treasury_yields.csv')

In [4]:
# Load FRED yields (20Y, 30Y)
yields = pd.read_csv('../data/raw/treasury_yields.csv', parse_dates=['DATE']).set_index('DATE')

# Simple proxy “long bond” yield for TLT
yields['tlt_yield'] = (0.5 * yields['yield_20y'] + 0.5 * yields['yield_30y']) / 100

In [8]:
yields.head()

,yield_20y,yield_30y,tlt_yield
DATE,,,
2018-01-01,NaN,NaN,NaN
2018-01-02,2.64,2.81,0.02725
2018-01-03,2.62,2.78,0.02700
2018-01-04,2.62,2.79,0.02705
2018-01-05,2.64,2.81,0.02725


Apprixmate bond prices

In [10]:
DURATION = 17.5  # years, average duration of TLT holdings

yields['tlt_price_proxy'] = 100 * np.exp(-yields['tlt_yield'] * DURATION)

yields.head()

,yield_20y,yield_30y,tlt_yield,tlt_price_proxy
DATE,,,,
2018-01-01,NaN,NaN,NaN,NaN
2018-01-02,2.64,2.81,0.02725,62.072011
2018-01-03,2.62,2.78,0.02700,62.344171
2018-01-04,2.62,2.79,0.02705,62.289644
2018-01-05,2.64,2.81,0.02725,62.072011


Convert bond prices to ETF NAV

In [12]:
SHARES_OUTSTANDING = 554_600_000  # from holdings file
AUM = 50e9  # approximate TLT AUM in USD, adjust as needed

# NAV = (AUM / Shares Outstanding) * (price_proxy / 100)
yields['nav_proxy'] = (AUM / SHARES_OUTSTANDING) * (yields['tlt_price_proxy'] / 100)

yields.head()

,yield_20y,yield_30y,tlt_yield,tlt_price_proxy,nav_proxy
DATE,,,,,
2018-01-01,NaN,NaN,NaN,NaN,NaN
2018-01-02,2.64,2.81,0.02725,62.072011,55.961063
2018-01-03,2.62,2.78,0.02700,62.344171,56.206429
2018-01-04,2.62,2.79,0.02705,62.289644,56.157270
2018-01-05,2.64,2.81,0.02725,62.072011,55.961063


In [22]:
etf = pd.read_csv('../data/raw/tlt_daily.csv')

# Use the exact column as present: "Date"
etf['Date'] = pd.to_datetime(etf['Date'], errors='coerce', utc=True)

# Drop any bad rows
etf = etf.dropna(subset=['Date'])

# Convert from tz-aware -> tz-naive, then normalize to date
etf['DATE'] = etf['Date'].dt.tz_convert(None).dt.normalize()

# Set index to DATE and drop the old Date column to avoid confusion
etf = etf.drop(columns=['Date']).set_index('DATE')

# ---------- Yields: load & ensure nav_proxy ----------
yields = pd.read_csv('../data/raw/treasury_yields.csv', parse_dates=['DATE']).set_index('DATE')


# Normalize to date-only to match ETF index
yields.index = yields.index.normalize()

# If nav_proxy not present yet, build it quickly from your earlier proxy columns:
if 'nav_proxy' not in yields.columns:
    # Require tlt_yield and tlt_price_proxy; build them if missing
    if 'tlt_yield' not in yields.columns:
        # blend 20y/30y (adjust weights if you like)
        yields['tlt_yield'] = (0.5*yields['yield_20y'] + 0.5*yields['yield_30y'])/100.0
    if 'tlt_price_proxy' not in yields.columns:
        DURATION = 17.5
        yields['tlt_price_proxy'] = 100.0*np.exp(-yields['tlt_yield']*DURATION)

    # Scale to per-share NAV
    AUM = 50_000_000_000.0        # replace with actual AUM if you have it
    SHARES = 554_600_000.0        # from your holdings snapshot
    yields['nav_proxy'] = (AUM/SHARES) * (yields['tlt_price_proxy']/100.0)

# ---------- Join & compute basis ----------
combined = etf.join(yields[['nav_proxy']], how='inner')

# Sanity checks
print('ETF index dtype:', combined.index.dtype)
print('Combined head:\n', combined.head())

# Basis in fraction and bps
combined['basis'] = (combined['etf_close'] - combined['nav_proxy']) / combined['nav_proxy']
combined['basis_bps'] = combined['basis'] * 1e4

# Save for downstream notebooks
combined.to_csv('../data/processed/tlt_nav_basis.csv')

ETF index dtype: datetime64[ns]
Combined head:
               etf_open    etf_high     etf_low   etf_close  etf_volume  \
DATE                                                                     
2020-11-09  134.091588  134.246333  132.939558  134.168961    23245000   
2020-11-10  133.223321  133.954081  133.068563  133.395264    12561000   
2020-11-11  133.274827  134.065774  133.231838  133.876633     5888200   
2020-11-12  134.925535  136.180729  134.727801  136.154938    15999700   
2020-11-13  136.146338  136.266700  135.759464  135.974396     7344600   

            nav_proxy  
DATE                   
2020-11-09  67.899668  
2020-11-10  67.662435  
2020-11-11        NaN  
2020-11-12  68.977554  
2020-11-13  68.856949  
